In [1]:
# Methods by David Rushing Dewhurt, drd@davidrushingdewhurst.com
import Pkg
Pkg.activate("..")

using CrimsonSkyline
using Distributions: Normal, truncated, Poisson, InverseWishart, MvNormal, Dirichlet, Categorical
using Logging
using StatsBase: mean, std
using Random: seed!
using PrettyPrint: println
using CSV
using DataFrames

seed!(2021)

MersenneTwister(2021)

In [2]:
df = DataFrame(CSV.File("clustering_tbl.csv"))

,Walking Score,Standing Score,Fall Status,Subject Ind
,Float64,Float64,Int64,String
1,0.969173,0.683472,1,S0001
2,0.97548,0.181886,1,S0001
3,0.973374,0.218314,1,S0001
4,0.98586,0.683472,1,S0001
5,0.999961,0.120885,1,S0001
6,0.957081,0.46664,1,S0001
7,4.43399e-5,0.687266,2,S0002
8,0.0037153,0.318062,2,S0002
9,8.08068e-5,0.372071,2,S0002


(465, 2)

In [10]:
data = hcat(df[:,1],df[:,2])

465×2 Matrix{Float64}:
 0.969173     0.683472
 0.97548      0.181886
 0.973374     0.218314
 0.98586      0.683472
 0.999961     0.120885
 0.957081     0.46664
 4.43399e-5   0.687266
 0.0037153    0.318062
 8.08068e-5   0.372071
 0.000261917  0.57516
 0.000557769  0.697527
 0.0318466    0.160412
 2.08222e-5   0.473805
 ⋮            
 2.70189e-6   0.163626
 2.69162e-5   0.624352
 1.2263e-5    0.163626
 5.48441e-5   0.517141
 0.00413509   0.289526
 0.0129474    0.624352
 0.000625765  0.68496
 0.00134128   0.68496
 0.000420474  0.687814
 5.87997e-5   0.120885
 0.00029688   0.517141
 0.000836797  0.373165

In [4]:
function eye(d :: Int)
    mat = zeros(Float64, d, d)
    for i in 1:d
        mat[i, i] = 1.0
    end
    mat
end

eye (generic function with 1 method)

In [13]:
function clustering(t :: Trace, data :: Array{Float64, 2}, cluster_guess :: Int)
    # data has shape (D, N), where D is dimensionalty and N is num observations
    D = size(data, 2)
    N = size(data, 1)
    # first, choosing the number of clusters. Prior penalizes high number of clusters
    cluster_guess = max(cluster_guess, 1)
    n_clusters = sample(t, :n_clusters, truncated(Poisson(cluster_guess), 1, Inf))
    # second, given the number of clusters, define loc and cov for each
    covs = Array{Array{Float64, 2}, 1}()
    locs = Array{Array{Float64, 1}, 1}()
    id_mat = eye(D)
    for c in 1:n_clusters
        cov = sample(t, (:cov, c), InverseWishart(D + 1, id_mat))
        push!(covs, id_mat)
        loc = sample(t, (:loc, c), MvNormal(zeros(D), cov))
        push!(locs, loc)
    end
    # third, for each datapoint, assign to a cluster
    # note that each datapoint has its own local rv -- we could introduce an amortized model
    # that learns a mapping from datapoint features to cluster
    for n in 1:N
        cluster_prob = sample(t, (:cluster_prob, n), Dirichlet(n_clusters, 1.0))
        this_cluster = sample(t, (:this_cluster, n), Categorical(cluster_prob))
        observe(t, (:data, n), MvNormal(locs[this_cluster], covs[this_cluster]), data[n, :])
    end
end

clustering (generic function with 1 method)

In [14]:
function main()
    @info "Open-universe clustering inference"
#     D = 4
#     n1 = 5
#     n2 = 12
#     data1 = rand(D, n1)
#     data2 = rand(D, n2) .+ 6
#     data = hcat(data1, data2)
#     @info "True number of clusters is 2"
    # pretend we don't know dgp, bias it to be higher than ground truth
    n_clusters_guess = 4
    @time results = mh(clustering; params = (data, n_clusters_guess), burn=2000, thin=100, num_iterations=100000)
    mean_n_clusters = mean(results, :n_clusters)
    std_n_clusters = std(results, :n_clusters)
    @info "posterior n_clusters ≈ $mean_n_clusters ± $std_n_clusters"
end

main()

┌ Info: Open-universe clustering inference
└ @ Main In[14]:2


294.324552 seconds (3.72 G allocations: 166.825 GiB, 10.67% gc time, 0.05% compilation time)


┌ Info: posterior n_clusters ≈ 2.0 ± 0.0
└ @ Main In[14]:15


LoadError: UndefVarError: results not defined